# Entrega laboratorio 3

## Integrantes 

- Javier Steven Barrera Toro - 202214779
- Julian Santiago Rolon Toloza - 202215839

In [1]:
from matplotlib import pyplot as plt
from typing import List, Tuple, NewType
import numpy as np
import pandas as pd
import sympy as sp
import copy
import tabulate

np.seterr(divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Problema 1: Implementación del Método Simplex Estándar

Una solución básica factible del siguiente problema de optimización puede ser $(x_1, x_2, x_3) = (0, 0, 0)$ ya que cumple con las restricciones. 

\begin{equation}
\begin{aligned}
\textrm{Maximizar} \quad & Z = 3x_1 + 2x_2 + 5x_3 \\
\textrm{s.t.} \quad & x_1 + x_2 + x_3 \leq 100 \\
  & 2x_1 + x_2 + x_3 \leq 150 \\ 
  & x_1 + 4x_2 + 2x_3 \leq 80 \\
  & x_1, x_2, x_3 \geq 0
\end{aligned}
\end{equation}

## Convertir el problema a la forma estándar

Para convertir el problema anterior en su forma estándar se deben agregar variables de holgura a las restricciones de tal manera que las mismas se vuelvan igualdades. Además, se agregan estas variables a la restricción de no negatividad y a la función objetivo.

\begin{equation}
\begin{aligned}
\textrm{Maximizar} \quad & Z = 3x_1 + 2x_2 + 5x_3 + 0s_1 + 0s_2 + 0s_3 \\
\textrm{s.t.} \quad & x_1 + x_2 + x_3 + s_1 = 100 \\
  & 2x_1 + x_2 + x_3 + s_2 = 150 \\ 
  & x_1 + 4x_2 + 2x_3 + s_3 = 80 \\
  & x_1, x_2, x_3, s_1, s_2, s_3 \geq 0
\end{aligned}
\end{equation}

## Implementación del algoritmo del método Simplex

In [131]:
class SimplexSolver:
    def __init__(self, c, X, b, variables):
        self.z = np.array(c)
        self.X = np.array(X)
        self.b = np.array(b)
        self.variables = variables
        self.unlimited = False
 
    def solve(self):
        row_num, col_num = self.X.shape
        simplex_table = np.vstack([self.z, self.X])
        c = np.hstack([[0], self.b]).reshape((row_num + 1, 1))
        simplex_table = np.hstack([simplex_table, c], dtype='float64')
        
        aux_vars = len(self.z) - 1 - self.variables
        rows = ['z'] + ['s_'+str(i+1) for i in range(aux_vars)]
        cols = ['z'] + \
            ['x_'+str(i+1) for i in range(self.variables)] + \
            ['s_'+str(i+1) for i in range(aux_vars)] + ['solution']

        history = [simplex_table.copy()]
        state = [[rows, cols]]

        while True:
            if np.min(simplex_table[0]) >= 0: break

            pivot_col = np.argmin(simplex_table[0])
            reduced_costs = simplex_table[1:, -1] / simplex_table[1:, pivot_col]

            if np.max(reduced_costs) <= 0:
                print("UNLIMITED_PROBLEM: There's no reduced costs greater than 0")
                self.unlimited = True
                break

            valid_values = reduced_costs[(reduced_costs > 0)]
            min_value = valid_values.min()
            pivot_row = np.where(reduced_costs == min_value)[0][0] + 1
            
            element = simplex_table[pivot_row, pivot_col]
            simplex_table[pivot_row] = simplex_table[pivot_row, :] / element

            for i in range(simplex_table.shape[0]):
                if i != pivot_row:
                    term = simplex_table[i, pivot_col]
                    simplex_table[i] = simplex_table[i, :] - term * simplex_table[pivot_row, :]
            
            # update rows and cols of simplex table
            current_state = copy.deepcopy(state[-1])
            new_rows, new_cols = current_state
            out, inp = new_rows[pivot_row], new_cols[pivot_col]

            new_rows[pivot_row] = inp      # update rows
            new_cols[pivot_col] = out      # update cols
            state.append([new_rows, new_cols])
            
            history.append(simplex_table.copy())

        # construct the solution
        solution = [0 for _ in range(self.variables)]
        maps = {f'x_{i+1}': i for i in range(self.variables)}
        # iterate over the rows to get the solutions
        lr_state, _ = state[-1]
        for col, val_col in enumerate(lr_state):
            value = maps.get(val_col, None)
            if value != None:
                solution[value] = simplex_table[:, -1][col]
        
        optimal_value = simplex_table[0, -1]
        
        self.history = history
        self.states = state

        return np.array(solution), optimal_value

    def print_iterations(self):
        if self.unlimited:
            print('UNLIMITED_PROBLEM')
            return
        
        for i, obj in enumerate(zip(self.states, self.history)):
            state, table = obj
            row_headers, col_headers = state
            print(f'Iteración {i+1}:')
            
            # Convertir a lista y agregar encabezado de fila
            table_with_row_headers = [
                [row_headers[i]] + row.tolist() for i, row in enumerate(table)
            ]
            
            # Encabezados completos (columna vacía + encabezados de columna)
            full_headers = ['Basic'] + col_headers
            
            to_print =tabulate.tabulate(
                table_with_row_headers,
                headers=full_headers,
                tablefmt='fancy_grid'
            )

            print(to_print, end='\n'*2)


In [145]:
c = [1, -3, -2, -5, 0, 0, 0]
b = [100, 150, 80]
X = [
    [0, 1, 1, 1, 1, 0, 0],
    [0, 2, 1, 1, 0, 1, 0],
    [0, 1, 4, 2, 0, 0, 1]
]

solver = SimplexSolver(c, X, b, 3)
solution, optimal_value = solver.solve()
print(solution, optimal_value)

[73.33333333  0.          3.33333333] 236.66666666666666


In [146]:
solver.print_iterations()

Iteración 1:
╒═════════╤═════╤═══════╤═══════╤═══════╤═══════╤═══════╤═══════╤════════════╕
│ Basic   │   z │   x_1 │   x_2 │   x_3 │   s_1 │   s_2 │   s_3 │   solution │
╞═════════╪═════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪════════════╡
│ z       │   1 │    -3 │    -2 │    -5 │     0 │     0 │     0 │          0 │
├─────────┼─────┼───────┼───────┼───────┼───────┼───────┼───────┼────────────┤
│ s_1     │   0 │     1 │     1 │     1 │     1 │     0 │     0 │        100 │
├─────────┼─────┼───────┼───────┼───────┼───────┼───────┼───────┼────────────┤
│ s_2     │   0 │     2 │     1 │     1 │     0 │     1 │     0 │        150 │
├─────────┼─────┼───────┼───────┼───────┼───────┼───────┼───────┼────────────┤
│ s_3     │   0 │     1 │     4 │     2 │     0 │     0 │     1 │         80 │
╘═════════╧═════╧═══════╧═══════╧═══════╧═══════╧═══════╧═══════╧════════════╛

Iteración 2:
╒═════════╤═════╤═══════╤═══════╤═══════╤═══════╤═══════╤═══════╤════════════╕
│ Basic   │   z │   x_1 │

# Problema 2: Implementación del método Simplex Dual Phase

En esta sección se nos pide implementar el método Simplex de Dos Fases para minimizar una función objetivo. Recordemos que $\min Z$ es equivalente a $- \max (-Z)$. Por tanto, dado el siguiente problema de optimización se realizará la anterior conversión.

\begin{equation}
\begin{aligned}
\textrm{Minimizar} \quad & Z = 5x_1 - 4x_2 + 3x_3 \\
\textrm{sujeto a} \quad & 2x_1 + x_2 - x_3 = 10 \\
  & x_1 - 3x_2 + 2x_3 \geq 5 \\ 
  & x_1 + x_2 + x_3 \leq 15 \\
  & x_1, x_2, x_3 \geq 0
\end{aligned}
\end{equation}

## Transformar el problema 

Lo cual es equivalente al siguiente problema de optimización:

\begin{equation}
\begin{aligned}
\textrm{Maximizar} \quad & -Z = -5x_1 + 4x_2 - 3x_3 \\
\textrm{sujeto a} \quad & 2x_1 + x_2 - x_3 = 10 \\
  & x_1 - 3x_2 + 2x_3 \geq 5 \\ 
  & x_1 + x_2 + x_3 \leq 15 \\
  & x_1, x_2, x_3 \geq 0
\end{aligned}
\end{equation}

Una vez tenemos este problema de maximización podemos transformarlo a su forma estandar.

\begin{equation}
\begin{aligned}
\textrm{Maximizar} \quad & -Z = -5x_1 + 4x_2 - 3x_3 + 0s_1 + 0s_2 \\
\textrm{sujeto a} \quad & 2x_1 + x_2 - x_3 = 10 \\
  & x_1 - 3x_2 + 2x_3 - s_1 = 5 \\ 
  & x_1 + x_2 + x_3 + s_2 = 15 \\
  & x_1, x_2, x_3, s_1, s_2 \geq 0
\end{aligned}
\end{equation}

## Implementación del algoritmo del método Simplex de Dos Fases

In [5]:
class DualPhaseSimplexMethodSolver:
    def __init__(self, c, X, b):
        self.c = c
        self.X = X
        self.b = b

    def phase1(self):
        pass

## Fase I

## Fase II

## Solución óptima y el valor de la función objetivo

# Problema 3: Comparación de rendimiento con GLPK/Pyomo

Dijo que comparar el número de iteraciones.

# Problema 4: Análisis de sensibilidad en programación lineal 

Se desea realizar un análisis de sensibilidad sobre la solución óptima obtenida mediante el método simplex para el problema siguiente.

$$
\begin{equation}
\begin{aligned}
\textrm{Maximizar} \quad & Z = 4x_1 + 3x_2 \\
\textrm{sujeto a} \quad & x_1 + 2x_2 \leq 8 \\
  & 3x_1+ 2x_2 \leq 12 \\
  & x_1, x_2 \geq 0
\end{aligned}
\end{equation}
$$

In [142]:
class SimplexSolverWithSensitiveAnalysis(SimplexSolver):
    def __init__(self, c, X, b, variables):
        super().__init__(c, X, b, variables)

    def sensitivity_analysis(self):
        # todas las variables basicas son las de la columna
        # las variables no basicas estan en fila 1 columna[1:self.variables+1]
        last_table = self.history[-1]
        first_table = self.history[0]
        first_state = self.states[0]
        last_state = self.states[-1]

        variables = {f'x_{i+1}' for i in range(self.variables)}
        n_slack_variables = first_table[0].shape[0] - (self.variables + 2)
        s_variables = {f's_{i+1}' for i in range(n_slack_variables)}
        
        basic_variables = []
        non_basic_variables = []
        for i, variable in enumerate(last_state[0][1:]):
            value = last_table[i+1, -1]
            if value:
                basic_variables.append(variable)
            elif value == 0:
                non_basic_variables.append(variable)
        non_basic_variables += last_state[1][1:self.variables+1]
        
        # non-basic variables
        print('NON-BASIC VARIABLES')
        coefficients_non_basic = {}
        for non_basic_variable in non_basic_variables:
            cj_actual = 0 # if actual is 'x_i' it's zero
            if non_basic_variable.startswith('s'):
                # search actual in solution column
                index = first_state[0].index(non_basic_variable)
                cj_actual = first_table[:, -1][index]

            if non_basic_variable.startswith('s'):
                index = last_state[1][1+self.variables:].index(non_basic_variable)
                cj_opt = last_table[0, 1 + self.variables+index]
            else:
                index = last_state[1][:self.variables+1].index(non_basic_variable)
                cj_opt = last_table[0, index]
                
            coefficients_non_basic[non_basic_variable] = cj_opt
            print(f"c_j <= {cj_actual} + {cj_opt} = {cj_actual + cj_opt}")

        # basic variables
        print('\nBASIC VARIABLES')
        for basic_variable in basic_variables:
            index_basic = last_state[0].index(basic_variable)
            li = []
            ls = []
            c_actual = 0 # if actual c is 0 if it's a slack variable
            for non_basic_variable in non_basic_variables:
                if non_basic_variable.startswith('s'):
                    index = 1+self.variables+last_state[1][1+self.variables:].index(non_basic_variable)
                elif non_basic_variable.startswith('x'):
                    index = last_state[1].index(non_basic_variable)
                    j = int(non_basic_variable.split('_')[1])
                    c_actual = -first_table[0, j]
                    
                cj = last_table[0, index]
                aij = last_table[index_basic, index]

                if aij > 0:
                    li.append(cj / aij)
                if aij < 0:
                    ls.append(cj / aij)

            l = cj - min(li, default=np.inf)
            h = cj + max(ls, default=np.inf)
            print(f'VARIABLES ({basic_variable}): {l} <= c_i <= {h}')


In [143]:
c = [1, -30, -20, 0, 0]

X = [
    [0, 2, 1, 1, 0],
    [0, 1, 3, 0, 1]
]

b = [8, 8]

simplex = SimplexSolverWithSensitiveAnalysis(c, X, b, 2)
solution, _min = simplex.solve()
print(solution, _min)
simplex.sensitivity_analysis()

[3.2 1.6] 128.0
NON-BASIC VARIABLES
c_j <= 8.0 + 14.0 = 22.0
c_j <= 8.0 + 2.0 = 10.0

BASIC VARIABLES
VARIABLES (x_1): -21.333333333333336 <= c_i <= -8.0
VARIABLES (x_2): -3.0 <= c_i <= -68.0


In [144]:
simplex.print_iterations()

Iteración 1:
╒═════════╤═════╤═══════╤═══════╤═══════╤═══════╤════════════╕
│ Basic   │   z │   x_1 │   x_2 │   s_1 │   s_2 │   solution │
╞═════════╪═════╪═══════╪═══════╪═══════╪═══════╪════════════╡
│ z       │   1 │   -30 │   -20 │     0 │     0 │          0 │
├─────────┼─────┼───────┼───────┼───────┼───────┼────────────┤
│ s_1     │   0 │     2 │     1 │     1 │     0 │          8 │
├─────────┼─────┼───────┼───────┼───────┼───────┼────────────┤
│ s_2     │   0 │     1 │     3 │     0 │     1 │          8 │
╘═════════╧═════╧═══════╧═══════╧═══════╧═══════╧════════════╛

Iteración 2:
╒═════════╤═════╤═══════╤═══════╤═══════╤═══════╤════════════╕
│ Basic   │   z │   s_1 │   x_2 │   s_1 │   s_2 │   solution │
╞═════════╪═════╪═══════╪═══════╪═══════╪═══════╪════════════╡
│ z       │   1 │     0 │  -5   │  15   │     0 │        120 │
├─────────┼─────┼───────┼───────┼───────┼───────┼────────────┤
│ x_1     │   0 │     1 │   0.5 │   0.5 │     0 │          4 │
├─────────┼─────┼───────┼───